In [39]:
import numpy as np
import psycopg2
import cPickle

tags_raw = cPickle.load(open('../tags_classifier/data/tags_raw.p', 'rb'))
tags_norm = cPickle.load(open('../tags_classifier/data/tags_norm.p', 'rb'))
osago_table = cPickle.load(open('../tags_classifier/data/tags_table.p', 'rb'))

print len(set(osago_table.tag.values)- set(tags_raw))
print len(set(osago_table.norm.values)- set(tags_norm))

tags_table.head()

28
28


,tag,parent,support,answer,leaf,norm
0,Документы в СК,ОСАГО,документ страховой компания осаго;какой докуме...,"Так, вы хотели бы утонить список документов дл...",False,документ ск
1,причинении вреда имуществу потерпевшего,Документы в СК,список документ осаго причинение вредить имуще...,NaN,True,причинение вред имущество потерпевший
2,причинен ущерб здоровью,Документы в СК,возмещение ущерб здоровье осаго;осаго здоровье...,NaN,True,причинить ущерб здоровье
3,утрата профессиональной трудоспособности,Документы в СК,утрата трудоспособность дтп;потеря трудоспособ...,NaN,True,утрата профессиональный трудоспособность
4,причинения вреда жизни потерпевшего,Документы в СК,осаго смерть потерпевший;выплата осаго случай ...,NaN,True,причинение вред жизнь потерпевший


In [12]:
for t in tags_raw:
    if u'осаго' in t or u'ОСАГО' in t:
        print t

неустойка по ОСАГО
выплаты по ОСАГО
ремонт по ОСАГО
выплаты Росгосстраха по ОСАГО
возмещение ущерба по ОСАГО
отказ в выплате по ОСАГО
ОСАГО
досудебная претензия по ОСАГО
иск в суд на страховую компанию по ОСАГО
расторжение договора страхования жизни при ОСАГО
страховой случай по ОСАГО
регресс по ОСАГО
выплаты по ОСАГО при ДТП
расторжение договора ОСАГО


In [14]:
common_tags = set(osago_table.tag.values) - (set(osago_table.tag.values)- set(tags_raw))
for t in common_tags:
    print t

неустойка по ОСАГО
выплаты по ОСАГО
ОСАГО
ремонт по ОСАГО
расторжение договора ОСАГО


In [15]:
conn = psycopg2.connect("host=host port=0000 dbname=dbname user=user password=password")
cur = conn.cursor()

cur.execute("SELECT * FROM Tags")
colnames = [desc[0] for desc in cur.description]
print colnames

cur.close()
conn.close()

['id', 'name', 'original_id', 'question', 'norm_field', 'parent_id', 'leaf', 'active', 'support_field', 'response_field']


In [34]:
s = "aba \
kaba"
print s

aba kaba


In [49]:
tag_to_id = dict()

conn = psycopg2.connect("host=host port=0000 dbname=dbname user=user password=password")
cur = conn.cursor()

cur.execute("SELECT * FROM Tags")

all_table = cur.fetchall()
for row in all_table:
    tag_to_id[unicode(row[1].decode('utf-8'))] = row[0]

cur.close()
conn.close()

In [109]:
last_id = max(tag_to_id.values())
print last_id

2688


In [110]:
conn = psycopg2.connect("host=host port=0000 dbname=dbname user=user password=password")
cur = conn.cursor()

cur.execute(u"DELETE FROM Tags WHERE id>2688;")

conn.commit()
cur.close()
conn.close()

In [111]:
def null(s):
    if type(s) is float:
        return 'NULL'
    elif s == u'Уточните ваш вопрос':
        return 'NULL'
    else:
        return s

In [112]:
conn = psycopg2.connect("host=host port=0000 dbname=dbname user=user password=password")
cur = conn.cursor()

def get_tag_id(name):
    cur.execute(u"SELECT id FROM Tags WHERE name = '{0}'".format(name))
    return cur.fetchone()[0]

for row_ in osago_table.iterrows():
    row = row_[1]
    tag, parent = row.tag, row.parent
    parent_id = get_tag_id(parent) if type(parent) is not float else 'NULL'
    if tag in common_tags:
        cur.execute(u"UPDATE Tags SET norm_field = '{0}', parent_id = {1}, leaf = {2}, active = {3},\
 support_field = '{4}', response_field = '{5}'\
 WHERE name = '{6}';".format(row.norm, parent_id, row.leaf, True, null(row.support), null(row.answer), tag))
    else:
        last_id += 1
        cur.execute(u"INSERT INTO Tags (id, name, norm_field, parent_id, leaf, active, support_field, response_field)\
 VALUES ({0}, '{1}', '{2}', {3}, {4}, {5}, '{6}', '{7}');".\
            format(last_id, tag, row.norm, parent_id, row.leaf, True, null(row.support), null(row.answer), tag))

conn.commit()
cur.close()
conn.close()

In [113]:
conn = psycopg2.connect("host=ec2-54-163-248-14.compute-1.amazonaws.com port=5432 dbname=dd4p9sfsq52j3c\
 user=uhpdppnyewasnq password=1L6ffZTzONLVtjhibe4nlVjjL0")
cur = conn.cursor()

cur.execute(u"UPDATE Tags SET support_field=NULL WHERE support_field='NULL';")
cur.execute(u"UPDATE Tags SET response_field=NULL WHERE response_field='NULL';")

conn.commit()
cur.close()
conn.close()

In [118]:
conn = psycopg2.connect("host=ec2-54-163-248-14.compute-1.amazonaws.com port=5432 dbname=dd4p9sfsq52j3c\
 user=uhpdppnyewasnq password=1L6ffZTzONLVtjhibe4nlVjjL0")
cur = conn.cursor()

for row_ in osago_table.iterrows():
    row = row_[1]
    tag, parent = row.tag, row.parent

    cur.execute(u"SELECT * FROM Tags WHERE name = '{0}'".format(tag))
    for i in cur.fetchone():
        print i, i is None
    print 

cur.close()
conn.close()

2689 False
Документы в СК False
None True
None True
документ ск False
1700 False
False False
True False
документ страховой компания осаго;какой документ нужный страховой компания дтп;список документ страховой компания осаго False
Так, вы хотели бы утонить список документов для страховой компании после ДТП. Опишите, пожалуйста, подробней какой ущерб был нанесен. False

2690 False
причинении вреда имуществу потерпевшего False
None True
None True
причинение вред имущество потерпевший False
2689 False
True False
True False
список документ осаго причинение вредить имущество;причинение вред имущество потерпевший False
None True

2691 False
причинен ущерб здоровью False
None True
None True
причинить ущерб здоровье False
2689 False
True False
True False
возмещение ущерб здоровье осаго;осаго здоровье;вред здоровье дтп;оплата травма осаго;документ компенсация вред здоровье;компенсация травма дтп False
None True

2692 False
утрата профессиональной трудоспособности False
None True
None True
утрата

In [129]:
import pandas as pd

conn = psycopg2.connect("host=host port=0000 dbname=dbname user=user password=password")
cur = conn.cursor()

cur.execute("SELECT * FROM Tags WHERE active=True")
colnames = [desc[0] for desc in cur.description]

table = cur.fetchall()
df = pd.DataFrame(table, columns=colnames)

cur.close()
conn.close()

df.head()

,id,name,original_id,question,norm_field,parent_id,leaf,active,support_field,response_field
0,1700,ОСАГО,None,"Насколько мы поняли, у Вас возник вопрос по ОС...",осаго,NaN,False,True,None,None
1,119,неустойка по ОСАГО,None,None,неустойка осаго,362.0,True,True,расчёт неустойка осаго;посчитать неустойка оса...,None
2,2481,расторжение договора ОСАГО,None,None,расторжение договор осаго,2702.0,True,True,расторгнуть договор осаго;расторжение осаго;ра...,None
3,362,выплаты по ОСАГО,None,None,выплата осаго,1700.0,False,True,выплата осаго дтп;выплата осаго;возмещение вре...,"Так, у вас возник вопрос связанный с выплатами..."
4,646,ремонт по ОСАГО,None,None,ремонт осаго,1700.0,False,True,ремонт автомобиль осаго;стоимость ремонт осаго...,None


In [133]:
df.index = df.id.values

In [134]:
df.head()

,id,name,original_id,question,norm_field,parent_id,leaf,active,support_field,response_field
1700,1700,ОСАГО,None,"Насколько мы поняли, у Вас возник вопрос по ОС...",осаго,NaN,False,True,None,None
119,119,неустойка по ОСАГО,None,None,неустойка осаго,362.0,True,True,расчёт неустойка осаго;посчитать неустойка оса...,None
2481,2481,расторжение договора ОСАГО,None,None,расторжение договор осаго,2702.0,True,True,расторгнуть договор осаго;расторжение осаго;ра...,None
362,362,выплаты по ОСАГО,None,None,выплата осаго,1700.0,False,True,выплата осаго дтп;выплата осаго;возмещение вре...,"Так, у вас возник вопрос связанный с выплатами..."
646,646,ремонт по ОСАГО,None,None,ремонт осаго,1700.0,False,True,ремонт автомобиль осаго;стоимость ремонт осаго...,None


In [138]:
df.loc[:, 'norm_field'] = df.norm_field.apply(lambda s: unicode(s.decode('utf-8')))

In [153]:
df[df.norm_field==u'осаго'].iloc[0,:]

id                                                             1700
name                                                          ОСАГО
original_id                                                    None
question          Насколько мы поняли, у Вас возник вопрос по ОС...
norm_field                                                    осаго
parent_id                                                       NaN
leaf                                                          False
active                                                         True
support_field                                                  None
response_field                                                 None
Name: 1700, dtype: object